In [17]:
from pathlib import Path
import os
import pandas as pd
import pyperclip
import subprocess
from reload_recursive import reload_recursive
import helpers

import mri_data

In [2]:
reload_recursive(mri_data)
from mri_data import file_manager as fm
from mri_data import utils

pd.options.mode.copy_on_write = True

In [19]:
drive_root = fm.get_drive_root()
dataroot = drive_root / "3Tpioneer_bids"
labelroot = drive_root / "srs-9/3Tpioneer_bids_predictions"
data_dir = Path("/home/srs-9/Projects/ms_mri/analysis/paper1/data0")
df = pd.read_csv(data_dir / "manual_labels.csv")
df.set_index("subid", inplace=True)

df_full = pd.read_csv(data_dir / "t1_2_data_full.csv", index_col="subid")
df_full = helpers.set_dz_type5(df_full)

In [5]:
for i, row in df.iterrows():
    scanroot = dataroot / str(row['sub-ses']) 
    images = []
    for im in ["t1.nii.gz", "t1_gd.nii.gz", "flair.nii.gz"]:
        if (scanroot / im).is_file():
            images.append(str(scanroot / im))
    if len(images) == 0:
        continue
    labels = []
    for lab in ["pineal-SRS.nii.gz", "pineal-SRS_T1.nii.gz"]:
        if (scanroot / lab).is_file():
            labels.append(str(scanroot / lab))
    
    cmd = utils.open_itksnap_workspace_cmd(images, labels, win=False)
    df.loc[i, "cmd"] = cmd

In [ ]:
sub = 1010
cmd = df.loc[sub, "cmd"]
print(cmd)
pyperclip.copy(cmd)
# subprocess.Popen(cmd.split(" "))

KeyError: 1487

In [9]:
for i, row in df_full.iterrows():
    scanroot = dataroot / str(row['sub-ses']) 
    images = []
    for im in ["t1.nii.gz", "t1_gd.nii.gz", "flair.nii.gz"]:
        if (scanroot / im).is_file():
            images.append(str(scanroot / im))
    if len(images) == 0:
        continue
    labels = []
    for lab in ["pineal-SRS.nii.gz", "pineal-SRS_T1.nii.gz"]:
        if (scanroot / lab).is_file():
            labels.append(str(scanroot / lab))
    
    cmd = utils.open_itksnap_workspace_cmd(images, labels, win=True)
    df_full.loc[i, "cmd"] = cmd

In [24]:
man_pineal_df = pd.DataFrame(columns=df_full.columns)
for i, row in df_full.iterrows():
    scanroot = dataroot / str(row['sub-ses']) 
    if (scanroot / "pineal-SRS.nii.gz").is_file() or (scanroot / "pineal-SRS_T1.nii.gz").is_file():
        man_pineal_df.loc[i, :] = row

In [21]:
man_pineal_df['dz_type5']

1001     RMS
1010     RMS
1011     RMS
1019     RMS
1029    NIND
1033     RMS
1036     RMS
1037     RMS
1038     RMS
1057     RMS
1065     RMS
1066     PMS
1080     RMS
1085     RMS
1087     RMS
1089     RMS
1097     RMS
1103     PMS
1109     RMS
1117     PMS
1119     RMS
1152     RMS
1163    OIND
1188     RMS
1191    OIND
1234     RMS
1248     PMS
1259     RMS
1265     RMS
1272     RMS
1280    NIND
1293     RMS
1321    NIND
1328     PMS
1355    NIND
1437    NIND
1453     PMS
1486    NIND
1498    OIND
1518    OIND
1540    OIND
1544     RMS
1547    NIND
1548    NIND
2081    OIND
2083    NIND
2097    NIND
2126    NIND
2132    OIND
2144    NIND
2146    NIND
2164    OIND
2187    OIND
Name: dz_type5, dtype: object

In [11]:
sub = 1066
cmd = df_full.loc[sub, "cmd"]
print(cmd)
pyperclip.copy(cmd)
# subprocess.Popen(cmd.split(" "))

itksnap -g H:/3Tpioneer_bids/sub-ms1066/ses-20180828/t1.nii.gz -o H:/3Tpioneer_bids/sub-ms1066/ses-20180828/t1_gd.nii.gz H:/3Tpioneer_bids/sub-ms1066/ses-20180828/flair.nii.gz -s H:/3Tpioneer_bids/sub-ms1066/ses-20180828/pineal-SRS_T1.nii.gz


In [23]:
df_check = df_full[["pineal_volume", "dz_type5"]]
df_check.sort_values(by="pineal_volume", inplace=True)